# Optimize your Christmas shopping route using OpenRouteService

It's Christmas time again and everyone is busy buying presents and visiting christmas markets. In order to get all those tasks done in a most efficient way, you want to get the optimal route connecting all your stops. The Open Route Service will help us finding that route. 

First, we need to import some python modules.

In [1]:
import folium
from shapely import wkt, geometry
import json
import os.path

## 1. Create a list of all the stops on your Christmas shopping route
We will use the Nominatim geocoder to find the coordinates of POIs that are mapped in OSM. Use your email address as "user_agent".

In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Christina.ludwig@uni-heidelberg.de")

In [3]:
# Not important: Function that switches the x and y coordinates, so that it is displayed correctly in the map
def switchCoords(coords):
    return (coords[1], coords[0])

In [28]:
stops = [{'coordinates' : switchCoords(geolocator.geocode("Wochenmarkt, Heidelberg, Germany")[1]), 
        'name': 'Christmas Market'},
        {'coordinates' : switchCoords(geolocator.geocode("Universitäts-Apotheke, Heidelberg, Germany")[1]), 
        'name': 'Pharmacy'},
        {'coordinates' : switchCoords(geolocator.geocode("Bismarckplatz, Heidelberg, Germany")[1]), 
        'name': 'Bismarkplatz'},
        {'coordinates' : switchCoords(geolocator.geocode("Schwarzer Peter, Heidelberg, Germany")[1]), 
        'name': 'Schwarzer Peter'},
        {'coordinates' : switchCoords(geolocator.geocode("Zeughaus, Heidelberg, Germany")[1]), 
        'name': 'Marstall'},
        {'coordinates' : switchCoords(geolocator.geocode("Heidelberger Zuckerladen, Heidelberg, Germany")[1]), 
        'name': 'Zuckerladen'}
        ]

In [29]:
stops

[{'coordinates': (8.71047601790124, 49.41212685), 'name': 'Christmas Market'},
 {'coordinates': (8.7045353, 49.4114141), 'name': 'Pharmacy'},
 {'coordinates': (8.69299861340895, 49.41001065), 'name': 'Bismarkplatz'},
 {'coordinates': (8.6845637, 49.4032452), 'name': 'Schwarzer Peter'},
 {'coordinates': (8.70515231355789, 49.4129347), 'name': 'Marstall'},
 {'coordinates': (8.7031265, 49.4093992), 'name': 'Zuckerladen'}]

## 2. Create a map that shows all stops

Now we are going to visualize the stops of our Christmas route. The center the map should be on the Wochenmarkt. Therfore, we extract the coordinates of the first element in the list `stops`. Aftwards, we create a map object and display it.


In [30]:
centerOfMap = geometry.Point(stops[0]['coordinates'])
# Create a map object
mapHD = folium.Map(tiles='Stamen Toner',location=(centerOfMap.y, centerOfMap.x), zoom_start=14)
# Display updated map
mapHD

Now, let's add all stops as markers to the map.

In [31]:
stopMarkers = []
for stop in stops:
    lon, lat = stop['coordinates']
    name = stop['name']
    popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
    icon = folium.map.Icon(color='lightgray',
                        icon_color='#b5231a',
                        icon='star', # fetches font-awesome.io symbols
                        prefix='fa')
    folium.map.Marker([lat, lon], icon=icon, popup=popup).add_to(mapHD)
    stopMarkers.append(name)
# Display updated map
mapHD

## 3. Find the optimal route connecting all stops
### 3.1 Find optimal order of stops
We will use different ORS tools to determine the optimal route between the stops. Therefore, we need to import all necessary ORS modules.

In [32]:
from openrouteservice import client, distance_matrix, directions
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In order to use the OpenRouteService API you have to create an account and request a free API key [here](https://openrouteservice.org/dev/#/login). Paste your API key below into the variable "api_key". Using this key, we create a client object (`myClient`) hat will execute all your requests to the ORS.

In [33]:
api_key = '5b3ce3597851110001cf62483ae5cf98ae4743649600ff877943b339' #Provide your personal API key
ORSclient = client.Client(key=api_key)

In order to find the optimal route, we first calculate the distances between all route stops. Set the routing profile (e.g. foot-walking, driving-car) and metric.

In [34]:
# ORS routing profile
profile = 'foot-walking'
# Metric used to optimize route (duration or distance)
metric = ['duration']

In [35]:
# Extract the coordinates from the list of stops
stop_coords = [feat['coordinates'] for feat in stops]
# Request to the ORS 
request = {'locations': stop_coords,
           'profile': profile,
           'metrics': metric}
# Execute the request
stops_matrix = ORSclient.distance_matrix(**request)

The result of the request is a matrix that contains the distances between all stops. For better readability we will visualize it as a dataframe using the pandas module.

In [36]:
import pandas as pd
# Stored data from stops_matrix in dataframe object
stop_names = [st['name'] for st in stops]
stops_dataframe = pd.DataFrame(stops_matrix['durations'], columns=stop_names, index=stop_names)
# Print the data frame
stops_dataframe

,Christmas Market,Pharmacy,Bismarkplatz,Schwarzer Peter,Marstall,Zuckerladen
Christmas Market,0.00,345.77,1014.06,1839.72,406.19,570.61
Pharmacy,345.77,0.00,668.30,1502.28,165.83,252.74
Bismarkplatz,1014.06,668.30,0.00,835.54,800.05,629.67
Schwarzer Peter,1839.72,1502.28,835.54,0.00,1605.88,1269.59
Marstall,406.19,165.83,800.05,1605.88,0.00,411.07
Zuckerladen,570.61,252.74,629.67,1269.59,411.07,0.00


The code below identifies the optimal order of stops. Don't look at it in too much detail. It's not necessary to understand all of it 


In [37]:
def getDistance(from_id, to_id):
    return int(stops_matrix['durations'][from_id][to_id])

tsp_size = len(stop_names)
num_routes = 1
start = 0 # arbitrary start location

optimal_coords = []

if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    routing.SetArcCostEvaluatorOfAllVehicles(getDistance)
    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start) # Index of the variable for the starting node.
        route = ''
        # while not routing.IsEnd(index):
        for node in range(routing.nodes()):
            optimal_coords.append(stop_coords[routing.IndexToNode(index)])
            route += str(stop_names[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        route += str(stop_names[routing.IndexToNode(index)])
        optimal_coords.append(stop_coords[routing.IndexToNode(index)])
        print("Route:\n" + route)

Total duration: 65.11666666666666 minutes

Route:
Christmas Market -> Marstall -> Bismarkplatz -> Schwarzer Peter -> Zuckerladen -> Pharmacy -> Christmas Market


### 3.2 Calculate route between stops
Now that we have the optimal order of stops, we only need to find a route connecting them. To see how much time we saved, we will compare it to the route which a random order of stops. 

Calculate the random route 

In [39]:
stop_coords.append(stop_coords[0])

In [40]:
request = {'coordinates': stop_coords,
           'profile': profile,
           'geometry': 'true',
           'format_out': 'geojson',
          }
random_route = ORSclient.directions(**request)

Calculate the optimal route. 

In [41]:
# Replace the coordinates in the request by the coordinates with the optimal order of stops
request['coordinates'] = optimal_coords
optimal_route = ORSclient.directions(**request)

Now, let's visualize everything in a map.

In [42]:
# Create map object
mapHD = folium.Map(tiles='Stamen Toner',location=(centerOfMap.y, centerOfMap.x), zoom_start=14)

# Just a function to style the routes on the map
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)

# Add random route to map
folium.features.GeoJson(data=random_route,
                        name='Random Route',
                        style_function=style_function('#84e184'),
                       overlay=True).add_to(mapHD)

# Add optimal route to map
folium.features.GeoJson(data=optimal_route,
                        name='Optimal Route',
                        style_function=style_function('#6666ff'),
                       overlay=True).add_to(mapHD)

# Add markers for each stop to map
stopMarkers = []
for stop in stops:
    lon, lat = stop['coordinates']
    name = stop['name']
    popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
    icon = folium.map.Icon(color='lightgray',
                        icon_color='#b5231a',
                        icon='star', # fetches font-awesome.io symbols
                        prefix='fa')
    folium.map.Marker([lat, lon], icon=icon, popup=popup).add_to(mapHD)
    stopMarkers.append(name)

# Add the layer control so you can toggle the routes on and off
mapHD.add_child(folium.LayerControl())
# Display the map
mapHD

Let's calculate how much time we save, if we optimize our Christmas shopping route.

In [43]:
optimal_duration = optimal_route['features'][0]['properties']['summary'][0]['duration'] / 60
random_duration = random_route['features'][0]['properties']['summary'][0]['duration'] / 60 
print("Duration optimal route: {0:.3f} mins\nDuration random route: {1:.3f} mins".format(optimal_duration,
                                                                                         random_duration))

Duration optimal route: 65.167 mins
Duration random route: 73.955 mins


## 4. Export the optimal route to a geojson file
The last step is to export the route to a geojson file.

In [19]:
import json
with open("/Users/chludwig/Data/temp/optimalRoute.geojson", "w") as fp: 
    json.dump(optimal_route, fp)

## Exercises

__Exercise 0:__ Create an OpenRouteService account and request an API key.

__Exercise 1:__ Go through the notebook and execute all code blocks. Try to understand what is happing in each one of them. Just focus on identifying the **main processing steps** without looking too much into the details of the code. Draw a flowchart that contains all input parameters (list of stops, routing profile and metric), output data sets / parameters and the main processing steps that connect them. You can use for example [draw.io](http://draw.io)

__Exercise 2__: Adapt the list of stops to your personal Christmas shopping route. These are the tasks that you need to compelete:
* buy presents at three different stores
* drink Glühwein at one of the Christmas markets
* get some food
* go to a pharmacy because you feel a cold coming up

Execute the whole script and analyse the results.

__Exercise 3:__ Try different routing profiles and metrics and compare your results. For a list of available profiles see [Open Route Service Matrix Tool](https://openrouteservice.org/documentation/#/reference/matrix/matrix-service-(get))